<a href="https://colab.research.google.com/github/itazap/MIE424-FinalProject/blob/main/MIE424_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIE424 Project

## 1. Load the Adult dataset

In [1]:
!git clone https://github.com/mlohaus/SearchFair.git
%cd SearchFair

Cloning into 'SearchFair'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 86 (delta 32), reused 74 (delta 24), pack-reused 0
Unpacking objects: 100% (86/86), done.
/Users/Chris/Documents/School/Current/MIE424/Final Project/MIE424-FinalProject/SearchFair


In [218]:
import pandas as pd
import time

In [236]:
# Load data into pandas DataFrame
dataset = pd.read_csv('data/adult/adult.csv')

# Drop fnlwgt, education, education-num, capital-gain, capital-loss as Lohaus et al do
dataset = dataset.drop(columns=['fnlwgt', 'education', 'capital-gain', 'capital-loss'])
dataset

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,13,Divorced,Prof-specialty,Not-in-family,White,Female,36,United-States,<=50K
48838,64,?,9,Widowed,?,Other-relative,Black,Male,40,United-States,<=50K
48839,38,Private,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,50,United-States,<=50K
48840,44,Private,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,40,United-States,<=50K


In [237]:
def replaceWithOneHot(df, col_name):
    # Takes in a pandas dataframe and replaces column with name col_name
    # with multiple columns of its one-hot encoding
    one_hots = pd.get_dummies(dataset[col_name], prefix=col_name)
    df = df.drop(columns =[col_name])
    df = df.join(one_hots)
    return df 

## Onehot categorical variables
#for feature in ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']:
#    dataset = replaceWithOneHot(dataset, feature);

#Format below features in dataset to be binary based on Lohaus' get_real_data.py
dataset.loc[dataset['age'] > 37, 'age'] = 1
dataset.loc[dataset['age'] != 1, 'age'] = 0

dataset.loc[dataset['workclass'] == 'Private', 'workclass'] = 1
dataset.loc[dataset['workclass'] != 1, 'workclass'] = 0

dataset.loc[dataset['education-num'] == 9, 'education-num'] = 1
dataset.loc[dataset['education-num'] != 1, 'education-num'] = 0

dataset.loc[dataset['marital-status'] == "Married-civ-spouse", 'marital-status'] = 1
dataset.loc[dataset['marital-status'] != 1, 'marital-status'] = 0

dataset.loc[dataset['occupation'] == "Craft-repair", 'occupation'] = 1
dataset.loc[dataset['occupation'] != 1, 'occupation'] = 0

dataset.loc[dataset['relationship'] == "Not-in-family", 'relationship'] = 1
dataset.loc[dataset['relationship'] != 1, 'relationship'] = 0

dataset.loc[dataset['race'] == "White", 'race'] = 1
dataset.loc[dataset['race'] != 1, 'race'] = 0

dataset.loc[dataset['hours-per-week'] > 40, 'hours-per-week'] = 1
dataset.loc[dataset['hours-per-week'] != 1, 'hours-per-week'] = 0

dataset.loc[dataset['native-country'] == "United-States", 'native-country'] = 1
dataset.loc[dataset['native-country'] != 1, 'native-country'] = 0

#Replace 'Male' with 1 and 'Female' with 0 in sex column
dataset.loc[dataset['sex'] == 'Male', 'sex'] = 1
dataset.loc[dataset['sex'] == 'Female', 'sex'] = -1

#replace '>50K' with 1 and '<=50K' with 0 in income column
dataset.loc[dataset['income'] == '>50K', 'income'] = 1
dataset.loc[dataset['income'] == '<=50K', 'income'] = -1

dataset

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,1,0,0,0,0,1,1,1,0,1,-1
1,1,0,0,1,0,0,1,1,0,1,-1
2,1,1,1,0,0,1,1,1,0,1,-1
3,1,1,0,1,0,0,0,1,0,1,-1
4,0,1,0,1,0,0,0,-1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...
48837,1,1,0,0,0,1,1,-1,0,1,-1
48838,1,0,1,0,0,0,0,1,0,1,-1
48839,1,1,0,1,0,0,1,1,1,1,-1
48840,1,1,0,0,0,0,0,1,0,1,-1


In [238]:
# Lohaus uses a random 10000 points for training, validation, and the rest for testing
# Since adult.csv is already shuffled, use the first 10000 rows as training and rest as testing

train_dataset = dataset.head(10000)
test_dataset = dataset.tail(len(dataset) - 10000)

## 2. Implement baseline models

In [222]:
from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator
import sklearn.metrics.pairwise as kernels
from sklearn.metrics import confusion_matrix
import numpy as np
import cvxpy as cp
import random

In [242]:
class BaselineModel(BaseEstimator):
    def __init__(self, l2_beta=0.001, kernel='linear', gamma=None, loss_name='hinge', lambda_max=1, max_iter=3000, solver='SCS', verbose=False):

        self.l2_beta = l2_beta # Regularization parameter beta for the l2 regularization
        self.kernel = kernel # The SVM kernel to be used.. Options:['linear','rbf','poly']
        self.gamma = gamma # If kernel='rbf', gamma to be kernel width, If kernel='poly', gamma to be degree.
        self.loss_name = loss_name # Loss function to be used. Options:['hinge','logistic','squared','exponential']
        self.lambda_max = lambda_max # The max lambda value for the start of the binary search.
        self.max_iter = max_iter # The number of iterations.
        self.solver = solver # The solver to be used by cvxpy. Options:['SCS','ECOS'].
        self.verbose = verbose # If true, Overrides the default of hiding solver output.
        
    def fit(self, x_train, y_train):
        """Fits a baseline SVM model on the given training data.
        Parameters
        ----------
        x_train: numpy array
            The features of the training data with shape=(number_points,number_features).
        y_train: numpy array
            The class labels of the training data with shape=(number_points,).
      
        Returns
        ----------
        self: object
        """

        self.x_train = np.matrix(x_train)
        self.y_train = np.array(y_train)

        if self.verbose:
            print("Preprocessing...")
        
        self._preprocess()

        lambda_min, lambda_max = 0, self.lambda_max

        self._construct_problem()
                
        self._optimize()
            
        criterion = False
        
        return self

    def predict(self, x_test):
        """Predict the label of test data.
        Parameters
        ----------
        x_test: numpy array
            The features of the test data with shape=(number_points,number_features).
        Returns
        ----------
        y_hat: numpy array
            The predicted class labels with shape=(number_points,).
        """
        y_hat = np.dot(self.coef_, np.transpose(x_test))
        
        return np.sign(y_hat)

    def _preprocess(self):
        """Setting the attributes loss_func and kernel_function.
        """
        self.coef_ = None
        if self.loss_name == 'logistic':
            self.loss_func = lambda z: cp.logistic(-z)
        elif self.loss_name == 'hinge':
            self.loss_func = lambda z: cp.pos(1.0 - z)
        elif self.loss_name == 'squared':
            self.loss_func = lambda z: cp.square(-z)
        elif self.loss_name == 'exponential':
            self.loss_func = lambda z: cp.exp(-z)
        else:
            print('Using default loss: hinge loss.')
            self.loss_func = lambda z: cp.pos(1.0 - z)

        if self.kernel == 'rbf':
            self.kernel_function = lambda X, Y: kernels.rbf_kernel(X, Y, self.gamma)
        elif self.kernel == 'poly':
            self.kernel_function = lambda X, Y: kernels.polynomial_kernel(X, Y, degree=self.gamma)
        elif self.kernel == 'linear':
            self.kernel_function = lambda X, Y: kernels.linear_kernel(X, Y) + 1
        else:
            self.kernel_function = kernel

    def _construct_problem(self):
        """ Construct the cvxpy minimization problem.
        """

        # Variable to optimize
        self.params = cp.Variable((self.x_train.shape[1], 1))
        self.bias = cp.Variable()
         
        # Loss Function to Minimize (with Regularization)
        self.loss = (1 /self.x_train.shape[1]) * cp.sum(self.loss_func(cp.multiply(self.y_train.reshape(-1, 1), self.x_train @ self.params - self.bias))) + self.l2_beta * cp.square(cp.norm(self.params, 2))
        
        # Final Problem Formulization
        self.prob = cp.Problem(cp.Minimize(self.loss))

    def _optimize(self):
        """Conduct the optimization of the created problem by using ECOS or SCS
        with cvxpy. 
        """

        if self.solver == 'SCS':
            self.prob.solve(solver=cp.SCS, max_iters=self.max_iter, verbose=self.verbose, warm_start=True)
        elif self.solver == 'ECOS':
            try:
                self.prob.solve(solver=cp.ECOS, max_iters=self.max_iter, verbose=self.verbose, warm_start=True)
            except Exception as e:
                self.prob.solve(solver=cp.SCS, max_iters=self.max_iter, verbose=self.verbose, warm_start=True)
    
        self.coef_ = self.params.value.squeeze()

## 3. Implement Test Procedure

In [239]:
class TestProcedure():
    def __init__(self,train_dataset,test_dataset,target,model,sensitive_attribute):
        self.X_train = train_dataset.drop(target,1)
        self.y_train = np.array(train_dataset.income)

        self.X_test = test_dataset.drop(target,1)
        self.y_test = np.array(test_dataset.income)
        
        self.model = model
        
        self.sensitive_attribute = sensitive_attribute
        
    def BuildModel(self):
        start_time = time.time()
        
        self.model.fit(self.X_train,self.y_train)
        
        end_time = time.time()
        build_time = end_time - start_time
        
        return build_time
        
    def RunTests(self):
        build_time = self.BuildModel()
        predictions = self.model.predict(self.X_test)
        prediction_accuracy = np.equal(self.y_test, predictions).mean()
        
        ddp,deo = self.compute_fairness_measures(predictions, self.y_test ,self.sensitive_attribute)
        
        return [build_time,prediction_accuracy,(ddp,deo)]
        
    def compute_fairness_measures(self, y_predicted, y_true, sens_attr):
        """Compute value of demographic parity and equality of opportunity for given predictions.
        Parameters
        ----------
        y_predicted: numpy array
            The predicted class labels of shape=(number_points,).
        y_true: numpy array
            The true class labels of shape=(number_points,).
        sens_attr: numpy array
            The sensitive labels of shape=(number_points,).
        Returns
        ----------
        DDP: float
            The difference of demographic parity.
        DEO: float
            The difference of equality of opportunity.
        """
       
        positive_rate_prot = self.get_positive_rate(y_predicted[sens_attr==-1], y_true[sens_attr==-1])
        positive_rate_unprot = self.get_positive_rate(y_predicted[sens_attr==1], y_true[sens_attr==1])
        true_positive_rate_prot = self.get_true_positive_rate(y_predicted[sens_attr==-1], y_true[sens_attr==-1])
        true_positive_rate_unprot = self.get_true_positive_rate(y_predicted[sens_attr==1], y_true[sens_attr==1])
        DDP = positive_rate_unprot - positive_rate_prot
        DEO = true_positive_rate_unprot - true_positive_rate_prot

        return DDP, DEO

    def get_positive_rate(self, y_predicted, y_true):
        """Compute the positive rate for given predictions of the class label.
        Parameters
        ----------
        y_predicted: numpy array
            The predicted class labels of shape=(number_points,).
        y_true: numpy array
            The true class labels of shape=(number_points,).
        Returns
        ---------
        pr: float
            The positive rate.
        """

        tn, fp, fn, tp = confusion_matrix(y_true, y_predicted).ravel()
        pr = (tp+fp) / (tp+fp+tn+fn)
        return pr

    def get_true_positive_rate(self, y_predicted, y_true):
        """Compute the true positive rate for given predictions of the class label.
        Parameters
        ----------
        y_predicted: numpy array
            The predicted class labels of shape=(number_points,).
        y_true: numpy array
            The true class labels of shape=(number_points,).
        Returns
        ---------
        tpr: float
            The true positive rate.
        """
        tn, fp, fn, tp = confusion_matrix(y_true, y_predicted).ravel()
        tpr = tp / (tp+fn)
        return tpr
        

In [240]:
basemodel_1 = BaselineModel()
sensitive_attribute = np.array(test_dataset.sex)
baseline_1_tester = TestProcedure(train_dataset,test_dataset,'income',basemodel_1,sensitive_attribute)

In [241]:
baseline_1_test_results = baseline_1_tester.RunTests()

(10000, 1)
(10000, 10)
(10, 1)
()


ValueError: unknown is not supported